# The Lattice Boltzmann Method - 3D

This notebook uses the Lattice Boltzmann (LBM) Method for numerical simulation of fluid flow, writte in python and jax.

jax is used to jit (just-in-time) the code, which speeds it up significantly.

The code is adjusted from the the code presented by Machine Learning & Simulation (MLS) in 2D:
- [Youtube](https://www.youtube.com/watch?v=ZUXmO4hu-20&list=LL&index=1&ab_channel=MachineLearning%26Simulation)
- [Git](https://github.com/Ceyron/machine-learning-and-simulation/blob/main/english/simulation_scripts/lattice_boltzmann_method_python_jax.py)

It is recommended to watch that video first, because a lot of explanation of this method, the setup and syntax mentioned in that video and code will be skipped here.

The adjustments here are twofold: the code has been adjusted for 3D, and a little noise is implemented. The noise is applied to the input velocities and will make sure interesting features emerge quicker.

## Dependencies
Let's get started with importing all the relevant packages.

3D takes a lot longer, even with jitted code. If you are running this on Google Colab or have a GPU that's jax compatible, don't forget to turn it on!

In [ ]:
import numpy as np
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
from tqdm import tqdm

jax.config.update("jax_enable_x64", True) # Set to False for more free memory, but less accuracy
plt.style.use("dark_background")

# Setup of the domain
The fluid flow simulation we are preparing is a cylinder in fluid flow, with the aim to make a Karman Vortex Street behind the cylinder. The modelled domain is going to be 300 cells along the x-axis and 50 along the y- and z-axis. The fluid is going to flow into the domain from the left, where x = 0.

In below cell is also the viscosity defined. The viscosity determines the Reynolds number, which is a measure of the characteristics of the flow, and the relaxation $\omega$, which is a relaxation parameter for the LBM method. The inverse ($1/\omega$) is the relaxation time $\tau$.

In [ ]:
cylinder_radius = 6
ny = 50
nz = 50
nx = 300

KINEMATIC_VISCOSITY = 0.0025
HORIZONTAL_INFLOW_VELOCITY = 0.04

reynolds_number = (HORIZONTAL_INFLOW_VELOCITY * cylinder_radius) / KINEMATIC_VISCOSITY
RELAXATION_OMEGA = (1.0 / (3.0 * KINEMATIC_VISCOSITY + 0.5))

print('Reynolds number:', reynolds_number)

The way the cylinder is stored as an object is as a boolean 3D array. Why a cylinder, you may ask. Isn't that just a 2D problem? It is, but this way we can compare it to the 2D model by MLS. Feel free to add more interesting shapes.

In [ ]:
x = jnp.arange(nx)
y = jnp.arange(ny)
z = jnp.arange(nz)
X, Y, Z = jnp.meshgrid(x, y, z, indexing="ij")

radii = jnp.sqrt((X - nx//5)**2 + (Y - ny//2)**2)
obstacle_mask = radii < cylinder_radius

print('Top view:')
plt.imshow(obstacle_mask[:, :, nz//2].T)
plt.show()
print('\nFront view:')
plt.imshow(obstacle_mask[nx//5, :, :].T)
plt.show()
print('\nSide view:')
plt.imshow(obstacle_mask[:, ny//2, :].T)
plt.show()

## D3Q19 - LBM
For the 3D version of the LBM method there is choice between D3Q15, D3Q19 and D3Q27. Here we are going to model the D3Q19 version. D3 means we are going to model in 3 dimensions. Q19 means that we have 19 vectors in our lattice as per this image (from [here](https://www.researchgate.net/publication/290158292_An_introduction_to_Lattice-Boltzmann_methods)):

![picture](https://drive.google.com/uc?id=1PAB1skjiYuenaNFffzURdcGxulT0_jpv)

For the BGK method, you need to know the opposite lattice velocity.

For the 3D Zou/He scheme, you need to know the vertices that are on the left of the inflow (where x = 0, LEFT_VELOCITIES) and the vertices that are on x = 0 (YZ_VELOCITIES) to calculate the pressure and velocity of the particles going from x = 0 to the right (RIGHT_PARTICLES).

Also, the lattice weights are defined.

In [ ]:
N_DISCRETE_VELOCITIES = 19
LATTICE_INDICES =          jnp.array([ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9,10,11,12,13,14,15,16,17,18])
LATICE_VELOCITIES_X =      jnp.array([ 0, 1, 0,-1, 0, 0, 0, 1,-1,-1, 1, 1,-1,-1, 1, 0, 0, 0, 0])
LATICE_VELOCITIES_Y =      jnp.array([ 0, 0, 1, 0,-1, 0, 0, 1, 1,-1,-1, 0, 0, 0, 0, 1,-1,-1, 1])
LATICE_VELOCITIES_Z =      jnp.array([ 0, 0, 0, 0, 0, 1,-1, 0, 0, 0, 0, 1, 1,-1,-1, 1, 1,-1,-1])

OPPOSITE_LATTICE_INDICES = jnp.array([ 0, 3, 4, 1, 2, 6, 5, 9,10, 7, 8,13,14,11,12,17,18,15,16])

LATTICE_VELOCITIES = jnp.array([LATICE_VELOCITIES_X,
                                LATICE_VELOCITIES_Y,
                                LATICE_VELOCITIES_Z])



LATTICE_WEIGHTS = jnp.array([# rest particle
                             1/3,

                             # face-connected neighbors
                             1/18, 1/18, 1/18, 1/18, 1/18, 1/18,

                             # edge-connected neighbors
                             1/36, 1/36, 1/36, 1/36, 1/36, 1/36, 1/36, 1/36, 1/36, 1/36, 1/36, 1/36])

RIGHT_VELOCITIES = jnp.array([1, 7, 10, 11, 14])             # LATICE_VELOCITIES_X = 1
LEFT_VELOCITIES = jnp.array([3, 8, 9, 12, 13])               # LATICE_VELOCITIES_X =-1
YZ_VELOCITIES = jnp.array([0, 2, 4, 5, 6, 15, 16, 17, 18])   # LATICE_VELOCITIES_X = 0

#Methods of the SLBM

## Density

The get_density function is the implementation of:

$\rho = \sum_{i} f_i \quad$   (formula (3) from [here](https://github.com/bartdavids/LatticeBoltzmannNotebooks/blob/main/README.md))


From 2D to 3D, nothing changes in the code when it comes to computing the density. It still is simply summing over all the lattice velocities, for each velocity. Instead of the shape (nx, ny, 9), the shape of the discrete velocities is not (nx, ny, nz, 19), which still means summing over the final axis.

## Macrosocopic velocities

The function get_macroscopic_velocities is the implementation of:

$\mathbf{u} = \sum_{i} f_i e_{i}\quad$(formula (4) from [here](https://github.com/bartdavids/LatticeBoltzmannNotebooks/blob/main/README.md))

In the Einstein summation the new z-axis should be added. In 2D, the summation was: "NMQ,dQ->NMd", where N and M are axes indicatng the x- and y-axes. In 3D the z-axis is added as L: "NMLQ,dQ->NMLd" or the elipsoid operator (...) can be used. Now it doesn't matter if the input is a 2D or 3D array!

## Equilibrium discrete velocities

In the function get_equilibrium_discrete_velocities we apply:


$f_i^{eq} = w_i \rho \left(1 + \frac{\mathbf{u} e_{i}}{c_s^2} + \frac{(\mathbf{u} e_{i})^2}{2c_s^4} - \frac{\mathbf{u}^2}{2c_s^2}\right)\quad$(formula (2) from [here](https://github.com/bartdavids/LatticeBoltzmannNotebooks/blob/main/README.md))

The same adjustment to 3D using the Einstein summation for projecting the discrete velocities is necessary. "NMQ,dQ->NMd" becomes "...Q,dQ->...d".

Throughout this part and subsequent parts, the addition of axes (with jnp.newaxis, or None), transposing and setting over specific axes (with, for instance:
[:, :, :] or [...]) take the additional z-axis in consideration.


In [ ]:
def get_density(discrete_velocities):
    return jnp.sum(discrete_velocities, axis=-1)

def get_macroscopic_velocities(discrete_velocities, density):
    return jnp.einsum("...Q, dQ -> ...d", discrete_velocities, LATTICE_VELOCITIES) / density[..., jnp.newaxis]

def get_equilibrium_discrete_velocities(macroscopic_velocities, density):
    projected_discrete_velocities = jnp.einsum("dQ, ...d -> ...Q", LATTICE_VELOCITIES, macroscopic_velocities)
    macroscopic_velocity_magnitude = jnp.linalg.norm(macroscopic_velocities, axis=-1, ord=2)
    equilibrium_discrete_velocities = (jnp.einsum("..., Q -> ...Q", density, LATTICE_WEIGHTS) *
        (1 + 3 * projected_discrete_velocities + 9/2 * projected_discrete_velocities**2 -
        3/2 * macroscopic_velocity_magnitude[..., jnp.newaxis]**2
        )
    )
    return equilibrium_discrete_velocities

## The steps of the LBM
The 7 steps as given in the MLS video and code are not adjusted much. The same boundary conditions and order of the steps are used. Mostly it is adding the right axis in the right place. The notable exceptions:
- In step 3, in the Zou/He scheme, where the density at the inflow boundary is being determined. In 2D, this is done with the purely vertical lattice velocities. In 3D the entire plane where the lattive velocity over x equals 0 should be taken (variable YZ_VELOCITIES).
- In step 7, a for-loop is used to take into account 2D and 3D input.

In [ ]:
@jax.jit
def update(discrete_velocities_prev):
    # (1) Prescribe the outflow BC on the right boundary. Flow can go out, but not back in.
    discrete_velocities_prev = discrete_velocities_prev.at[-1, ..., LEFT_VELOCITIES].set(discrete_velocities_prev[-2, ..., LEFT_VELOCITIES])

    # (2) Determine macroscopic velocities
    density_prev = get_density(discrete_velocities_prev)
    macroscopic_velocities_prev = get_macroscopic_velocities(
        discrete_velocities_prev,
        density_prev)

    # (3) Prescribe Inflow Dirichlet BC using Zou/He scheme in 3D:
    # https://arxiv.org/pdf/0811.4593.pdf
    # https://terpconnect.umd.edu/~aydilek/papers/LB.pdf
    macroscopic_velocities_prev = macroscopic_velocities_prev.at[0, ..., :].set(VELOCITY_PROFILE[0, ..., :])
    lateral_densities = get_density(jnp.einsum('i...->...i', discrete_velocities_prev[0, ..., YZ_VELOCITIES]))
    left_densities = get_density(jnp.einsum('i...->...i', discrete_velocities_prev[0, ..., LEFT_VELOCITIES]))
    density_prev = density_prev.at[0, ...].set((lateral_densities + 2 * left_densities) /
                                                (1 - macroscopic_velocities_prev[0, ..., 0]))

    # (4) Compute discrete Equilibria velocities
    equilibrium_discrete_velocities = get_equilibrium_discrete_velocities(
       macroscopic_velocities_prev,
       density_prev)

    # (3) Belongs to the Zou/He scheme
    discrete_velocities_prev =\
          discrete_velocities_prev.at[0, ..., RIGHT_VELOCITIES].set(
              equilibrium_discrete_velocities[0, ..., RIGHT_VELOCITIES])

    # (5) Collide according to BGK
    discrete_velocities_post_collision = (discrete_velocities_prev - RELAXATION_OMEGA *
          (discrete_velocities_prev - equilibrium_discrete_velocities))

    # (6) Bounce-Back Boundary Conditions to enfore the no-slip
    for i in range(N_DISCRETE_VELOCITIES):
        discrete_velocities_post_collision = discrete_velocities_post_collision.at[obstacle_mask, LATTICE_INDICES[i]].set(
                                                      discrete_velocities_prev[obstacle_mask, OPPOSITE_LATTICE_INDICES[i]])


    # (7) Stream alongside lattice velocities
    discrete_velocities_streamed = discrete_velocities_post_collision
    for i in range(N_DISCRETE_VELOCITIES):
        discrete_velocities_streamed = discrete_velocities_streamed.at[..., i].set(
            jnp.roll(discrete_velocities_post_collision[..., i],
            LATTICE_VELOCITIES[:, i], axis = (0, 1, 2)))

    return discrete_velocities_streamed

## Ready to run!
Now we define the parameters for plotting and running and innitialize the discrete velocities.

Here, the noise magnitude and the noise field that is added to the in velocities are defined. This will make sure we do not have to wait overly long on intersting features!

And then we run!

In [ ]:
PLOT_EVERY_N_STEPS = 100
SKIP_FIRST_N_ITERATIONS = 0
N_ITERATIONS = 10_000
noise_magnitude = 0.01
key = jax.random.PRNGKey(0)
VELOCITY_PROFILE = jnp.zeros((nx, ny, nz, 3))
NOISE = jax.random.normal(key, (ny, nz)) * noise_magnitude
VELOCITY_PROFILE = VELOCITY_PROFILE.at[:, :, :, 0].set(HORIZONTAL_INFLOW_VELOCITY + NOISE)
discrete_velocities_prev = get_equilibrium_discrete_velocities(VELOCITY_PROFILE,
                                                               jnp.ones((nx, ny, nz)))

In [ ]:
def run(discrete_velocities_prev):
    for i in tqdm(range(N_ITERATIONS)):
        discrete_velocities_next = update(discrete_velocities_prev)
        discrete_velocities_prev = discrete_velocities_next

        if i % PLOT_EVERY_N_STEPS == 0 and i > SKIP_FIRST_N_ITERATIONS - PLOT_EVERY_N_STEPS:
            density = get_density(discrete_velocities_next)
            macroscopic_velocities = get_macroscopic_velocities(
                discrete_velocities_next,
                density)
            velocity_magnitude = jnp.linalg.norm(
                macroscopic_velocities,
                axis=-1,
                ord=2)
            fig = plt.figure(figsize = (15, 3))
            cont = plt.contourf(
                X[..., nz//2], Y[...,  nz//2],
                jnp.flip(velocity_magnitude[...,  nz//2], axis = 1),
                levels=50,
                cmap="inferno",
                vmin= 0., vmax = HORIZONTAL_INFLOW_VELOCITY*1.5)
            plt.colorbar(cont).set_label("Velocity Magnitude")
            plt.gca().add_patch(
                plt.Circle(
                    (nx//5, ny//2-1),
                    cylinder_radius,
                    color="darkgreen",)
            )

            plt.axis('scaled')
            plt.axis('off')
            plt.show()

    return
run(discrete_velocities_prev)